# Data Cleansing

## Splitting participants_price

There are some columns which have values separated by "|". It is interesting to split these into different rows (one per value) in order to train the model. Here is an example with participants_price, although there are others which have to be also considered.

In [212]:
import pandas as pd
df = pd.read_csv('interview_analysis_molecule_x_10mg_v1.tsv', sep='\t')

In [213]:
#Converting the dataset into vertical mode
df['participants'] = df['participants'].str.split('|')
df['participants_price'] = df['participants_price'].str.split('|')

# Create new column with tuples
df['combined'] = df.apply(lambda row: list(zip(row['participants'], row['participants_price'])), axis=1)

# Explode 'combined' column
df = df.explode('combined')

# Create separate columns from 'combined'
df['participants'], df['participants_price'] = zip(*df['combined'])

# Drop 'combined' column
df = df.drop(columns=['combined'])

df['participants'] = df['participants'].str.strip()
df['participants_price'] = df['participants_price'].str.strip()

## 1. Independent variable winner_price

In [214]:
df.head()

,contract_id,published_date,start_date,duration_extension,outcome,second_place_outcome,buyer,region,atc,duration,...,maximum_price_allowed,active_ingredient,pack_strength,participants,participants_price,published_date_month,winner,winner_price,second_place,second_place_price
0,1,2013-05-16,2013-05-24,10,won,NaN,buyer_1,region_1,C07AB07,39,...,0.10000,molecule_x,10mg,participants_16,0.0004,2013-05-01,participants_16,0.0004,NaN,NaN
1,2,2013-04-29,2013-06-21,0,won,NaN,buyer_2,region_2,C07AB07,48,...,63442.21000,molecule_x,10mg,participants_16,0.007,2013-04-01,participants_16,0.0070,NaN,NaN
2,3,2013-06-09,2014-08-14,6,won,lost,buyer_3,region_3,C07AB07,12,...,0.11635,molecule_x,10mg,participants_16,0.014,2013-06-01,participants_16,0.0140,participants_23,0.07071
2,3,2013-06-09,2014-08-14,6,won,lost,buyer_3,region_3,C07AB07,12,...,0.11635,molecule_x,10mg,participants_23,0.07071,2013-06-01,participants_16,0.0140,participants_23,0.07071
3,4,2013-05-04,2013-05-17,19,won,NaN,buyer_4,region_4,C07AB07,36,...,0.03000,molecule_x,10mg,participants_16,0.025,2013-05-01,participants_16,0.0250,NaN,NaN


As I dont have much time, I will just clean it in a not "nice" way. However, for theses cases I like to use pydantic as it simplifies data validation.

### 1.1 Errors (automatic discard)

#### 1.1.1 winner_price should be float

In [215]:
mask = df['winner_price'].apply(lambda x: isinstance(x, float))

In [216]:
df1 = df[mask]

In [217]:
df_incorrectos = df[~mask]
print(df_incorrectos)

Empty DataFrame
Columns: [contract_id, published_date, start_date, duration_extension, outcome, second_place_outcome, buyer, region, atc, duration, contract_type, sku, end_date_extension, participants_no, quantity_annual, quantity_total, maximum_price_allowed, active_ingredient, pack_strength, participants, participants_price, published_date_month, winner, winner_price, second_place, second_place_price]
Index: []

[0 rows x 26 columns]


In [218]:
df1.head()

,contract_id,published_date,start_date,duration_extension,outcome,second_place_outcome,buyer,region,atc,duration,...,maximum_price_allowed,active_ingredient,pack_strength,participants,participants_price,published_date_month,winner,winner_price,second_place,second_place_price
0,1,2013-05-16,2013-05-24,10,won,NaN,buyer_1,region_1,C07AB07,39,...,0.10000,molecule_x,10mg,participants_16,0.0004,2013-05-01,participants_16,0.0004,NaN,NaN
1,2,2013-04-29,2013-06-21,0,won,NaN,buyer_2,region_2,C07AB07,48,...,63442.21000,molecule_x,10mg,participants_16,0.007,2013-04-01,participants_16,0.0070,NaN,NaN
2,3,2013-06-09,2014-08-14,6,won,lost,buyer_3,region_3,C07AB07,12,...,0.11635,molecule_x,10mg,participants_16,0.014,2013-06-01,participants_16,0.0140,participants_23,0.07071
2,3,2013-06-09,2014-08-14,6,won,lost,buyer_3,region_3,C07AB07,12,...,0.11635,molecule_x,10mg,participants_23,0.07071,2013-06-01,participants_16,0.0140,participants_23,0.07071
3,4,2013-05-04,2013-05-17,19,won,NaN,buyer_4,region_4,C07AB07,36,...,0.03000,molecule_x,10mg,participants_16,0.025,2013-05-01,participants_16,0.0250,NaN,NaN


#### 1.1.2. Check NaN/Not null

In [219]:
import numpy as np

In [220]:
mask = df1['winner_price'].apply(lambda x: x is not np.nan)

In [221]:
df2 = df1[mask]

In [222]:
#Always appending the new incorrect data into a dataframe with all the wrong rows
df_incorrectos=pd.concat([df_incorrectos, df1[~mask]])

In [223]:
print(df_incorrectos)

Empty DataFrame
Columns: [contract_id, published_date, start_date, duration_extension, outcome, second_place_outcome, buyer, region, atc, duration, contract_type, sku, end_date_extension, participants_no, quantity_annual, quantity_total, maximum_price_allowed, active_ingredient, pack_strength, participants, participants_price, published_date_month, winner, winner_price, second_place, second_place_price]
Index: []

[0 rows x 26 columns]


In [224]:
df2.head()

,contract_id,published_date,start_date,duration_extension,outcome,second_place_outcome,buyer,region,atc,duration,...,maximum_price_allowed,active_ingredient,pack_strength,participants,participants_price,published_date_month,winner,winner_price,second_place,second_place_price
0,1,2013-05-16,2013-05-24,10,won,NaN,buyer_1,region_1,C07AB07,39,...,0.10000,molecule_x,10mg,participants_16,0.0004,2013-05-01,participants_16,0.0004,NaN,NaN
1,2,2013-04-29,2013-06-21,0,won,NaN,buyer_2,region_2,C07AB07,48,...,63442.21000,molecule_x,10mg,participants_16,0.007,2013-04-01,participants_16,0.0070,NaN,NaN
2,3,2013-06-09,2014-08-14,6,won,lost,buyer_3,region_3,C07AB07,12,...,0.11635,molecule_x,10mg,participants_16,0.014,2013-06-01,participants_16,0.0140,participants_23,0.07071
2,3,2013-06-09,2014-08-14,6,won,lost,buyer_3,region_3,C07AB07,12,...,0.11635,molecule_x,10mg,participants_23,0.07071,2013-06-01,participants_16,0.0140,participants_23,0.07071
3,4,2013-05-04,2013-05-17,19,won,NaN,buyer_4,region_4,C07AB07,36,...,0.03000,molecule_x,10mg,participants_16,0.025,2013-05-01,participants_16,0.0250,NaN,NaN


#### 1.1.3 Check certain range (>0 and < 1 i.e)

In [225]:
mask = df2['winner_price'].apply(lambda x: 0 <= x <= 1)

In [226]:
df3 = df2[mask]

In [227]:
df_incorrectos=pd.concat([df_incorrectos, df2[~mask]])
print(df_incorrectos)

Empty DataFrame
Columns: [contract_id, published_date, start_date, duration_extension, outcome, second_place_outcome, buyer, region, atc, duration, contract_type, sku, end_date_extension, participants_no, quantity_annual, quantity_total, maximum_price_allowed, active_ingredient, pack_strength, participants, participants_price, published_date_month, winner, winner_price, second_place, second_place_price]
Index: []

[0 rows x 26 columns]


In [228]:
df3.head()

,contract_id,published_date,start_date,duration_extension,outcome,second_place_outcome,buyer,region,atc,duration,...,maximum_price_allowed,active_ingredient,pack_strength,participants,participants_price,published_date_month,winner,winner_price,second_place,second_place_price
0,1,2013-05-16,2013-05-24,10,won,NaN,buyer_1,region_1,C07AB07,39,...,0.10000,molecule_x,10mg,participants_16,0.0004,2013-05-01,participants_16,0.0004,NaN,NaN
1,2,2013-04-29,2013-06-21,0,won,NaN,buyer_2,region_2,C07AB07,48,...,63442.21000,molecule_x,10mg,participants_16,0.007,2013-04-01,participants_16,0.0070,NaN,NaN
2,3,2013-06-09,2014-08-14,6,won,lost,buyer_3,region_3,C07AB07,12,...,0.11635,molecule_x,10mg,participants_16,0.014,2013-06-01,participants_16,0.0140,participants_23,0.07071
2,3,2013-06-09,2014-08-14,6,won,lost,buyer_3,region_3,C07AB07,12,...,0.11635,molecule_x,10mg,participants_23,0.07071,2013-06-01,participants_16,0.0140,participants_23,0.07071
3,4,2013-05-04,2013-05-17,19,won,NaN,buyer_4,region_4,C07AB07,36,...,0.03000,molecule_x,10mg,participants_16,0.025,2013-05-01,participants_16,0.0250,NaN,NaN


## 2. Relationship winner_price vs. other variable

### Winner_price should be <= than participants_price

In [229]:
mask = df3.apply(lambda row: float(row['winner_price']) <= float(row['participants_price']), axis=1)

In [230]:
df4 = df3[mask]

In [231]:
df_incorrectos=pd.concat([df_incorrectos, df3[~mask]])
print(df_incorrectos)

Empty DataFrame
Columns: [contract_id, published_date, start_date, duration_extension, outcome, second_place_outcome, buyer, region, atc, duration, contract_type, sku, end_date_extension, participants_no, quantity_annual, quantity_total, maximum_price_allowed, active_ingredient, pack_strength, participants, participants_price, published_date_month, winner, winner_price, second_place, second_place_price]
Index: []

[0 rows x 26 columns]


In [232]:
df4.head()

,contract_id,published_date,start_date,duration_extension,outcome,second_place_outcome,buyer,region,atc,duration,...,maximum_price_allowed,active_ingredient,pack_strength,participants,participants_price,published_date_month,winner,winner_price,second_place,second_place_price
0,1,2013-05-16,2013-05-24,10,won,NaN,buyer_1,region_1,C07AB07,39,...,0.10000,molecule_x,10mg,participants_16,0.0004,2013-05-01,participants_16,0.0004,NaN,NaN
1,2,2013-04-29,2013-06-21,0,won,NaN,buyer_2,region_2,C07AB07,48,...,63442.21000,molecule_x,10mg,participants_16,0.007,2013-04-01,participants_16,0.0070,NaN,NaN
2,3,2013-06-09,2014-08-14,6,won,lost,buyer_3,region_3,C07AB07,12,...,0.11635,molecule_x,10mg,participants_16,0.014,2013-06-01,participants_16,0.0140,participants_23,0.07071
2,3,2013-06-09,2014-08-14,6,won,lost,buyer_3,region_3,C07AB07,12,...,0.11635,molecule_x,10mg,participants_23,0.07071,2013-06-01,participants_16,0.0140,participants_23,0.07071
3,4,2013-05-04,2013-05-17,19,won,NaN,buyer_4,region_4,C07AB07,36,...,0.03000,molecule_x,10mg,participants_16,0.025,2013-05-01,participants_16,0.0250,NaN,NaN


### 2.2 winner_price <= maximum price allowed

In [233]:
mask = df4.apply(lambda row: row['winner_price'] <= row['maximum_price_allowed'], axis=1)

In [234]:
df5 = df4[mask]

In [235]:
df_incorrectos=pd.concat([df_incorrectos, df4[~mask]])
print(df_incorrectos)

Empty DataFrame
Columns: [contract_id, published_date, start_date, duration_extension, outcome, second_place_outcome, buyer, region, atc, duration, contract_type, sku, end_date_extension, participants_no, quantity_annual, quantity_total, maximum_price_allowed, active_ingredient, pack_strength, participants, participants_price, published_date_month, winner, winner_price, second_place, second_place_price]
Index: []

[0 rows x 26 columns]


## 3. Check other variables on their own or related to other

### Maximum allowed price in range 0<price<1

In [236]:
mask = df5['maximum_price_allowed'].apply(lambda x: 0 <= x <= 1)

In [237]:
df6 = df5[mask]

In [238]:
df_incorrectos=pd.concat([df_incorrectos, df5[~mask]])

In [239]:
print(df_incorrectos)

    contract_id published_date  start_date  duration_extension outcome  \
1             2     2013-04-29  2013-06-21                   0     won   
10           11     2014-08-05  2014-10-30                   0     won   
10           11     2014-08-05  2014-10-30                   0     won   
10           11     2014-08-05  2014-10-30                   0     won   
13           14     2017-10-20  2017-09-20                   3     won   
23           24     2020-10-11  2020-11-25                   0     won   
23           24     2020-10-11  2020-11-25                   0     won   

   second_place_outcome     buyer     region      atc  duration  ...  \
1                   NaN   buyer_2   region_2  C07AB07        48  ...   
10                 lost   buyer_2   region_2  C07AB07        36  ...   
10                 lost   buyer_2   region_2  C07AB07        36  ...   
10                 lost   buyer_2   region_2  C07AB07        36  ...   
13                  NaN  buyer_12  region_12  C

In [240]:
print(df6)

    contract_id published_date  start_date  duration_extension outcome  \
0             1     2013-05-16  2013-05-24                  10     won   
2             3     2013-06-09  2014-08-14                   6     won   
2             3     2013-06-09  2014-08-14                   6     won   
3             4     2013-05-04  2013-05-17                  19     won   
4             5     2013-12-18  2013-12-17                   6     won   
5             6     2013-12-06  2014-02-21                  14     won   
6             7     2014-02-24  2014-01-01                   3     won   
7             8     2014-03-08  2014-05-22                  10     won   
7             8     2014-03-08  2014-05-22                  10     won   
7             8     2014-03-08  2014-05-22                  10     won   
7             8     2014-03-08  2014-05-22                  10     won   
8             9     2014-03-18  2014-05-30                   6     won   
9            10     2014-09-07  2014-0

There are many other possible analysis, for example, checking that maximum_price is in a determined range.

## Log wrong data into file

In [241]:
from datetime import datetime

now = datetime.now()

timestamp_str = now.strftime("%Y_%m_%d%H_%M_%S")

df_incorrectos['execution_time'] = timestamp_str

filename = f"error_log_{timestamp_str}.csv"

df_incorrectos.to_csv(filename, mode='a', header=False)

## Log clean data into file

I will write the clean data into a csv file as an example, another possibility would be writing into an SQL table.

In [242]:
filename = f"clean_data_{timestamp_str}.csv"

df6.to_csv(filename, mode='a', header=True)

## Questions

As I am running out of time, I won´t be able to perform further cleansing. However, many other tasks can be carried out, for example: check dates format, check that the number of bids in participants_price = participants_no...

Furthermore, depending on the models which will be implemented, the data probably has to be adapted and the column participants_price converted into a vertical mode. That means, having one row for each bid in a contract instead of having one row per contract and all the bids in one cell, as that makes it impossible the model training.

### What manual tasks would you perform?

I would automate the whole process but including some data visualization and an error log, in order the give the analysts some insights about how the process is and checking visually with some graphs that the data is correct. It would be interesting to set some KPIs, in order to follow the process and the evolution in time. For example, if there is an NaN rate of 1NaN per 1000 rows and we set it as a KPI, we may see it increase from 1 to 50 and therefore suggests that date is coming with poor quality.

###  How would you improve this process long term and how would you build your roadmap?

As mentioned before, at the beginning it may be ok with just a python script but in the future some logging, data visualization and KPIs display should be developed in order to follow the process when it is deployed into production. To begin with, the data does not seem to be too difficult in order to set an automatic Python script which may run on a daily basis and uploads the clean data into an SQL table. As the data flow increases, it will be interesting to analyze cleansing and processing times and then set a strategy It may be enough with just one server running serverless or we may need multiprocessing and some optimization, but we will only know that as the project scalates.

###  Would you change anything if you would need to scale this process from a few SKU's to hundreds and thousands.

I guess, to begin with, the process can run locally on the analyst's computer and the data can be uploaded to the database. If the running times are way too long, first, the script has to be optimized in order to decrease execution times. However, if running times cannot be handled locally, it will be interesting to run the script/process on a server. Here we have two possibilities: running serverless or on a server. Depending on the company's infrastructure we may choose one or the other. If already some server is deployed and not running on full capacity, we may use it in order to optimize server usage. However, another possibility may be running it serverless (as AWS lambda functions) so that costs are optimized.